In [1]:
import pandas as pd
import numpy as np

from selenium.webdriver import Chrome
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException, TimeoutException,NoSuchFrameException, WebDriverException, UnexpectedAlertPresentException
from selenium.webdriver.common.keys import Keys # 스크롤

import time
import sys # 개수 업데이트
import datetime

from tqdm.notebook import tqdm

import re

In [2]:
# path 설정
driver = Chrome('C:/Users/User/공공 빅데이터 인턴십_공영홈쇼핑/chromedriver.exe')

## (1). 접속 -> 최신순

In [3]:
url = 'https://play.google.com/store/apps/details?id=com.pub.fm&hl=ko&showAllReviews=true'

In [4]:
driver.get(url)

#### 현재: 관련도순 -> 최신순으로 바꾸기 위한 더보기 버튼 클릭

In [5]:
more_button = driver.find_element_by_css_selector('div.MocG8c.UFSXYb.LMgvRb.KKjvXb')
time.sleep(1)
more_button.click()

#### 최신순 클릭

In [11]:
time.sleep(3)
lates_button = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/c-wiz/div[1]/div/div[2]/div[1]')
## 가끔씩 안먹을때가 있음 -> 안될때는 수동으로 최신순 클릭해야함.

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/c-wiz/div[1]/div/div[2]/div[1]"}
  (Session info: chrome=86.0.4240.183)


In [7]:
lates_button.click()

## (2). 크롤링

In [8]:
## 별점 개수
review_counts = driver.find_element_by_css_selector('span.AYi5wd.TBRnV').text
review_counts = re.sub(',', '', review_counts)
review_counts = int(review_counts)
review_counts

4797

In [21]:
## 원하는 날짜까지 스크롤하는 함수
def appstore_scrawl(start_date):
    batang = driver.find_element_by_css_selector('body')
    while True:
        dates = driver.find_elements_by_css_selector('div.d15Mdf.bAhLNe > div.xKpxId.zc7KVe > div.bAhLNe.kx8XBd > div > span.p2TkOb')
        dates_list = list(map(lambda x: datetime.datetime.strptime(x.text, '%Y년 %m월 %d일').date(), dates))
        
        sys.stdout.write('\rlast date is %s' %dates_list[-1]) # last date update
        sys.stdout.flush()
        time.sleep(0.5)

        if dates_list[-1] < start_date:
            break
        else:
            batang.send_keys(Keys.PAGE_DOWN)

            ## 리뷰 더보기 버튼 클릭
            try:
                more_button = driver.find_element_by_css_selector('span.CwaK9')
                more_button.click()
                continue
            except:
                pass       

In [20]:
## input: 시작 날짜 입력
start_date = datetime.date(2020,10,1)

appstore_scrawl(start_date)

last date is 2020-09-05

In [22]:
## 내용이 긴 경우 -> 전체보기 버튼 클릭
more_review_buttons = driver.find_elements_by_css_selector('button.LkLjZd.ScJHi.OzU4dc')
for more_review_button in more_review_buttons:
    try:
        more_review_button.click()
        time.sleep(1)
    except:
        pass

In [26]:
review_texts = driver.find_elements_by_css_selector('div.UD7Dzf')
dates = driver.find_elements_by_css_selector('div.d15Mdf.bAhLNe > div.xKpxId.zc7KVe > div.bAhLNe.kx8XBd > div > span.p2TkOb')
stars = driver.find_elements_by_xpath("//span[@class='nt2C1d']/div[@class='pf5lIe']/div[@role='img']")
likes = driver.find_elements_by_xpath("//div[@aria-label='이 리뷰가 유용하다는 평가를 받은 횟수입니다.']")   

print('글 길이: {}'.format(len(review_texts)))
print('날짜 길이: {}'.format(len(dates)))
print('평점 길이: {}'.format(len(stars)))
print('좋아요 길이: {}'.format(len(likes)))
# 길이가 모두 같아야 함

글 길이: 160
날짜 길이: 160
평점 길이: 160
좋아요 길이: 160


In [28]:
## 데이터프레임
review_dict = []

for i in tqdm(range(len(review_texts))):
    review_dict.append({
        'DATE' : dates[i].text,
        'STAR' : stars[i].get_attribute('aria-label'),
        'REVIEW' : review_texts[i].text,
        'LIKE' : likes[i].text
    })
review_df = pd.DataFrame(review_dict)

In [35]:
print('길이: ', len(review_df))
review_df.head()

길이:  160


,DATE,STAR,REVIEW,LIKE
0,2020년 11월 10일,별표 5개 만점에 1개를 받았습니다.,와 앱......10번은 해야 들어가지네 ㅋㅋㅋ 계속 먹통이라니요,
1,2020년 11월 10일,별표 5개 만점에 1개를 받았습니다.,회원가입후 로그인할려니 자꾸 아이디랑 비번 확인하라고해서 확인했더니 틀린게 없네요 ...,
2,2020년 11월 10일,별표 5개 만점에 5개를 받았습니다.,상품 선택과 대금결제가 매우 편리하다.,
3,2020년 11월 10일,별표 5개 만점에 1개를 받았습니다.,앱 실행후 홈황면 뜨면 먹통됩니다.,
4,2020년 11월 10일,별표 5개 만점에 1개를 받았습니다.,먹통현상 때문에 물건을 살수가 없네요,


## (3).데이터 전처리

In [36]:
## 날짜 형식으로 변환
review_df['DATE'] = pd.to_datetime(review_df['DATE'], format='%Y년 %m월 %d일', errors='raise')
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    160 non-null    datetime64[ns]
 1   STAR    160 non-null    object        
 2   REVIEW  160 non-null    object        
 3   LIKE    160 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 5.1+ KB


In [37]:
## 별점만 추출
review_df['STAR'] = review_df['STAR'].apply(lambda x: int(re.findall('[0-9]', x)[1]))
review_df.head()

,DATE,STAR,REVIEW,LIKE
0,2020-11-10,1,와 앱......10번은 해야 들어가지네 ㅋㅋㅋ 계속 먹통이라니요,
1,2020-11-10,1,회원가입후 로그인할려니 자꾸 아이디랑 비번 확인하라고해서 확인했더니 틀린게 없네요 ...,
2,2020-11-10,5,상품 선택과 대금결제가 매우 편리하다.,
3,2020-11-10,1,앱 실행후 홈황면 뜨면 먹통됩니다.,
4,2020-11-10,1,먹통현상 때문에 물건을 살수가 없네요,


In [38]:
## 리뷰
## 특수문자 제거
import re

#텍스트 정제 (특수기호 제거)
for i, document in enumerate(review_df.REVIEW):
    document = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', document) #특수기호 제거, 정규 표현식     
    document = re.sub('\n', ' ', document)
    document = re.sub('\r', ' ', document)
    review_df['REVIEW'][i] = document

<ipython-input-38-57c8038e6713>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df['REVIEW'][i] = document


In [77]:
## 좋아요
review_df['LIKE'] = review_df['LIKE'].fillna(0)
review_df['LIKE'].isna().sum()

0

In [78]:
## 연도, 월 추출
import datetime
review_df['year'] = review_df['DATE'].dt.year
review_df['month'] = review_df['DATE'].dt.month
review_df.head()

,DATE,STAR,REVIEW,LIKE,year,month
0,2020-11-10,1,와 앱10번은 해야 들어가지네 ㅋㅋㅋ 계속 먹통이라니요,,2020,11
1,2020-11-10,1,회원가입후 로그인할려니 자꾸 아이디랑 비번 확인하라고해서 확인했더니 틀린게 없네요 ...,,2020,11
2,2020-11-10,5,상품 선택과 대금결제가 매우 편리하다,,2020,11
3,2020-11-10,1,앱 실행후 홈황면 뜨면 먹통됩니다,,2020,11
4,2020-11-10,1,먹통현상 때문에 물건을 살수가 없네요,,2020,11


In [80]:
start_date = datetime.date(2020,10,1)
last_date = datetime.date(2020,10,31)

sel_df = review_df[review_df.DATE.apply(lambda x: x >= start_date and x <= last_date)].sort_values('DATE').reset_index(drop = True)
sel_df.head()

,DATE,STAR,REVIEW,LIKE,year,month
0,2020-10-01,1,무한 로그인시도뿐 살수가없네,,2020,10
1,2020-10-04,1,로그인이안됨 비밀번호재설정5번이나새로해도 안되서 재설치하러옴,,2020,10
2,2020-10-05,1,e쿠폰은 어디로 사라지게 하신건가요 복붙답변말고 이거에 대한 답좀 해주세요ㅋㅋ,,2020,10
3,2020-10-06,5,오랜만에 다시 사용하는데 어플이 아주 빨라졌네요 굿굿 별 다섯개 드립니다,6,2020,10
4,2020-10-07,5,상품이 믿을만하다,,2020,10


In [82]:
sel_df.to_excel('앱스토어_크롤링_{start}_{last}.xlsx'.format(start = start_date, last= last_date), index = False)

In [83]:
sel_df.to_csv('앱스토어_크롤링_{start}_{last}.csv'.format(start = start_date, last= last_date) , index = False)

In [88]:
## 최종 데이터프레임
review_df = pd.read_csv('앱스토어_크롤링_2020-10-01_2020-10-31.csv')
review_df.head()

,DATE,STAR,REVIEW,LIKE,year,month
0,2020-10-01,1,무한 로그인시도뿐 살수가없네,NaN,2020,10
1,2020-10-04,1,로그인이안됨 비밀번호재설정5번이나새로해도 안되서 재설치하러옴,NaN,2020,10
2,2020-10-05,1,e쿠폰은 어디로 사라지게 하신건가요 복붙답변말고 이거에 대한 답좀 해주세요ㅋㅋ,NaN,2020,10
3,2020-10-06,5,오랜만에 다시 사용하는데 어플이 아주 빨라졌네요 굿굿 별 다섯개 드립니다,6.0,2020,10
4,2020-10-07,5,상품이 믿을만하다,NaN,2020,10


## (4). 형태소 분석

* 형태소 분석기

In [86]:
import konlpy 
from konlpy.tag import Komoran  # 형태소 분석기 불러오기

In [87]:
######## 사용: 명사, 형용사, 동사 ##########
komoran = konlpy.tag.Komoran()

def get_words(text):
    clean_words = []
    try:
        for word in komoran.pos(text):
            if word[1] in ['NNG', 'VA', 'VV', 'NNP']: #명사, 형용사, 동사, 고유명사
                clean_words.append(word[0])
        clean_words = [word for word in clean_words if len(word) > 1]
        return clean_words
    except:
        return ["error"]

* 카테고리별 tdm

In [ ]:
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import glob #자료묶기

def category_tdm(df_column, folder_path):
    for column_text in tqdm(df_column.unique()):
        print(column_text)
        df = review_df[df_column == column_text]
        
        cv = CountVectorizer(max_features=1000, tokenizer=get_words)
        tdm = cv.fit_transform(df['REVIEW'])
        
        wordcount = pd.DataFrame({'text': column_text,'단어': cv.get_feature_names(),'빈도': tdm.sum(axis=0).flat}).sort_values('빈도', ascending = False)
        wordcount.to_excel('./{folder}/2_평점크롤링_tdm_{name}.xlsx'.format(folder = folder_path, name = column_text), index=False)

In [ ]:
category_tdm(review_df.year, 'tdm_연도별')  # 원하는 카테고리, 폴더명

* 카테고리별 tdm 묶고 전처리

In [ ]:
import glob #자료묶기

def data_processing(folder_path, category_name):
    # 자료 묶기
    files = glob.glob('./{folder}/*.xlsx'.format(folder = folder_path))
    dfs = (pd.read_excel(f, index_col=0) for f in files)
    all_data = pd.concat(dfs, axis = 0)
    df_tdm = all_data.reset_index()
    
    # 아이 -> 아이디
    df_tdm = df_tdm.replace('아이', '아이디')
    
    # 제거할 단어
    del_words = ['홈쇼핑', '쇼핑', '공영']
    df_tdm = df_tdm[~df_tdm['단어'].isin(del_words)]
    
    df_tdm.to_excel('tdm_{category}_merge.xlsx'.format(category = category_name), index = False)

In [ ]:
data_processing('tdm_연도별', '연도별') # 원하는 카테고리, 카테고리명

In [ ]:
df_tdm = pd.read_excel('tdm_연도별_merge.xlsx')
df_tdm.head()

## (5). wordcloud

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 
import matplotlib
from wordcloud import WordCloud

from matplotlib import font_manager, rc
plt.rc('font', family='NanumGothic')

In [ ]:
## word cloud
wordcloud = WordCloud(font_path='NanumGothic',
                      background_color='white',
                      colormap = "tab10", 
                      width=1500, height=1000)

In [ ]:
for word in df_tdm.text.unique():
    print(word)
    df = df_tdm[df_tdm['text'] == word]
    
    frequencies = df.set_index('단어').to_dict()['빈도']
    keyword = wordcloud.generate_from_frequencies(frequencies) 
    array = keyword.to_array()
    
    plt.figure(figsize = (30,30))
    plt.imshow(array) 
    plt.axis('off') 
    plt.show()

## (6). top20

In [ ]:
def top20(word):
    print(word)
    df = df_tdm[df_tdm['text'] == word]
    
    df2 = df.sort_values('빈도', ascending=False).reset_index(drop = True).iloc[:20,1:]
    df2.index = df2.index + 1
    display(df2)

In [ ]:
for word in tqdm(df_tdm.text.unique()):
    top20(word)